# Assignment 2

In [1]:
# All libraries required
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

2024-05-10 01:45:24.769801: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-10 01:45:25.162392: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-10 01:45:27.619836: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Tensorflow Version
print(tf.__version__)

2.16.1


In [3]:
# Allocate more memory - https://www.tensorflow.org/guide/gpu

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 4GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=4096)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


2024-05-10 01:45:31.508346: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-10 01:45:32.810886: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-10 01:45:32.810953: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-10 01:45:32.843943: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-10 01:45:32.844099: I external/local_xla/xla/stream_executor

In [4]:
# Get data - https://www.tensorflow.org/tutorials/load_data/images#load_data_using_a_keras_utility

# Parameters
batch_size = 125912
IMG_SIZE = (512, 512)
data_dir = './hagridset'
seed_no = 388741
validation_split = 0.3

# Dataset - https://stackoverflow.com/questions/66036271/splitting-a-tensorflow-dataset-into-training-test-and-validation-sets-from-ker

# Auto split to training set (70%)
training_data_set = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                       image_size = IMG_SIZE,
                                                       validation_split = validation_split,
                                                       subset = 'training',
                                                       seed = seed_no,
                                                       color_mode = 'grayscale',
                                                       shuffle = True)

# Auto split to validation set and test set (30%)
validation_data_set = tf.keras.utils.image_dataset_from_directory(data_dir,
                                                       image_size = IMG_SIZE,
                                                       validation_split = validation_split,
                                                       subset = 'validation',
                                                       seed = seed_no,
                                                       color_mode = 'grayscale',
                                                       shuffle = True)

# Split further to validation set (10%) and test set (20%)
validation_batches = tf.data.experimental.cardinality(validation_data_set)
test_data_set = validation_data_set.take((2 * validation_batches) // 3)
validation_data_set = validation_data_set.skip((2 * validation_batches) // 3)

Found 125912 files belonging to 18 classes.
Using 88139 files for training.
Found 125912 files belonging to 18 classes.
Using 37773 files for validation.


In [5]:
# Number of classes
num_classes = len(training_data_set.class_names)
num_classes

18

In [6]:
# Efficiency for data loading
AUTOTUNE = tf.data.AUTOTUNE

#train_data_set = training_data_set.cache().prefetch(buffer_size=AUTOTUNE)
#validation_data_set = validation_data_set.cache().prefetch(buffer_size=AUTOTUNE)
#test_data_set = test_data_set.cache().prefetch(buffer_size=AUTOTUNE)

## Model 1: Standard Configuration

In [7]:
from tensorflow.keras import layers

# Model building
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 1)))
model.add(layers.Conv2D(12, 3, strides=2, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(18, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(24, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(32, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(48, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(96, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(128, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Conv2D(192, 3, padding='same', activation='relu'))
model.add(layers.MaxPooling2D())
model.add(layers.Flatten())
model.add(layers.Dense(192, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

In [8]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 256, 256, 12)   │           120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 128, 128, 12)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 128, 128, 18)   │         1,962 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 64, 64, 18)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 64, 64, 24)     │         3,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 32, 32, 24)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 32, 32, 32)     │         6,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 16, 16, 48)     │        13,872 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 8, 8, 48)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 8, 8, 96)       │        41,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 4, 4, 96)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 4, 4, 128)      │       110,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 2, 2, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 2, 2, 192)      │       221,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 1, 1, 192)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 192)            │        37,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 18)             │         3,474 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 441,004 (1.68 MB)

 Trainable params: 441,004 (1.68 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy'],
)

In [10]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [11]:
history = model.fit(training_data_set, validation_data=validation_data_set, epochs=100, callbacks=[callback])

Epoch 1/100


I0000 00:00:1715302082.809984  130457 service.cc:145] XLA service 0x7f60f00120c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1715302082.810230  130457 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2024-05-10 01:48:02.981996: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-05-10 01:48:03.284042: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


   5/2755 ━━━━━━━━━━━━━━━━━━━━ 1:20 29ms/step - accuracy: 0.0223 - loss: 3.5607     

I0000 00:00:1715302091.572153  130457 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


2755/2755 ━━━━━━━━━━━━━━━━━━━━ 134s 45ms/step - accuracy: 0.1618 - loss: 2.5806 - val_accuracy: 0.5735 - val_loss: 1.3098
Epoch 2/100
2755/2755 ━━━━━━━━━━━━━━━━━━━━ 117s 42ms/step - accuracy: 0.6331 - loss: 1.1233 - val_accuracy: 0.7007 - val_loss: 0.9601
Epoch 3/100
2755/2755 ━━━━━━━━━━━━━━━━━━━━ 120s 43ms/step - accuracy: 0.7374 - loss: 0.8087 - val_accuracy: 0.7462 - val_loss: 0.7916
Epoch 4/100
2755/2755 ━━━━━━━━━━━━━━━━━━━━ 117s 42ms/step - accuracy: 0.7818 - loss: 0.6723 - val_accuracy: 0.7631 - val_loss: 0.7758
Epoch 5/100
2755/2755 ━━━━━━━━━━━━━━━━━━━━ 116s 42ms/step - accuracy: 0.8085 - loss: 0.5925 - val_accuracy: 0.7857 - val_loss: 0.7825
Epoch 6/100
2755/2755 ━━━━━━━━━━━━━━━━━━━━ 120s 43ms/step - accuracy: 0.8260 - loss: 0.5387 - val_accuracy: 0.7664 - val_loss: 0.7459
Epoch 7/100
2755/2755 ━━━━━━━━━━━━━━━━━━━━ 118s 43ms/step - accuracy: 0.8367 - loss: 0.5021 - val_accuracy: 0.7801 - val_loss: 0.8009
Epoch 8/100
2755/2755 ━━━━━━━━━━━━━━━━━━━━ 119s 43ms/step - accuracy: 0.84

In [12]:
model.evaluate(test_data_set)

787/787 ━━━━━━━━━━━━━━━━━━━━ 22s 28ms/step - accuracy: 0.8200 - loss: 0.6272


[0.6337441205978394, 0.8193297386169434]

## Model 2: With Data Augmentation

In [13]:
data_augmentation = tf.keras.Sequential(
  [
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

In [14]:
model2 = tf.keras.models.Sequential()
model2.add(tf.keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 1)))
model2.add(data_augmentation)
model2.add(layers.Conv2D(12, 3, strides=2, padding='same', activation='relu'))
model2.add(layers.MaxPooling2D())
model2.add(layers.Conv2D(18, 3, padding='same', activation='relu'))
model2.add(layers.MaxPooling2D())
model2.add(layers.Conv2D(24, 3, padding='same', activation='relu'))
model2.add(layers.MaxPooling2D())
model2.add(layers.Conv2D(48, 3, padding='same', activation='relu'))
model2.add(layers.MaxPooling2D())
model2.add(layers.Conv2D(96, 3, padding='same', activation='relu'))
model2.add(layers.MaxPooling2D())
model2.add(layers.Conv2D(192, 3, padding='same', activation='relu'))
model2.add(layers.MaxPooling2D())
model2.add(layers.Flatten())
model2.add(layers.Dense(192, activation='relu'))
model2.add(layers.Dense(num_classes, activation='softmax'))

In [15]:
model2.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_1 (Sequential)       │ (None, 512, 512, 1)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 256, 256, 12)   │           120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 128, 128, 12)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 128, 128, 18)   │         1,962 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 64, 64, 18)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 64, 64, 24)     │         3,912 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 32, 32, 24)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 32, 32, 48)     │        10,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 16, 16, 48)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 16, 16, 96)     │        41,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 8, 8, 96)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 8, 8, 192)      │       166,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 4, 4, 192)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 3072)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 192)            │       590,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 18)             │         3,474 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 817,548 (3.12 MB)

 Trainable params: 817,548 (3.12 MB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model2.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [17]:
#history2 = model2.fit(training_data_set, validation_data=validation_data_set, epochs=100, callbacks=[callback])